# Reconhecimento de Gesto Dinâmicos Utilizando Redes Neurais Profundas


<br/>**Disciplina**: Introdução a redes neurais profundas 2017/1
<br/>**Professor**: Jorge Leonid Aching Samatelo
<br/>**Alunos**: Clebeson Canuto dos Santos e Leonardo de Assis Silva
    

## Códigos

In [ ]:
from __future__ import division, print_function
import tensorflow as tf
import numpy as np
import os

In [ ]:
max_steps = 8e4

initial_learning_rate = 1e-3
decay_steps = 7e2
decay_rate = 0.98

summary_step = 1e2
batch_size = 256

checkpoint_dir="./checkpoints"

n_classes = 10
rows, cols, channels = 100, 100, 3
input_file = './train.tfrecords'

In [ ]:
def read_decode_distort(queue):
    reader = tf.TFRecordReader()
    _, serialized = reader.read(queue)
    features = tf.parse_single_example(serialized, features={
      'image_raw': tf.FixedLenFeature([rows*cols*channels], tf.string),
      'label': tf.FixedLenFeature([], tf.int64),
    })
    
    label = tf.cast(features['label'], tf.int32)

    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image = tf.cast(image, tf.float32) 
    image = tf.reshape(image, [rows, cols, channels])
    
    #image = tf.image.resize_image_with_crop_or_pad(image, 96, 96)
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_brightness(image, max_delta=63)
    #image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
    
    image = tf.image.per_image_standardization(image)
    return image, label

def load_inputs(filename, batch_size):
    with tf.name_scope('input'):
        queue = tf.train.string_input_producer([filename])
        
        image, label = read_decode_distort(queue)
        
        image_batch, label_batch = tf.train.shuffle_batch(
            [image, label], 
            batch_size = batch_size, 
            num_threads = 4, 
            capacity = 1000 + 3*batch_size, 
            min_after_dequeue = 1000)
    
    #tf.summary.image('images', image_batch)
    label_batch = tf.one_hot(label_batch, n_classes)
    return image_batch, label_batch

In [ ]:
with tf.device('/cpu:0'):
    images, labels = load_inputs(input_file, batch_size)
    #images = tf.image.grayscale_to_rgb(tf.expand_dims(source_images[:,:,:,0], 3))

keep = tf.placeholder(tf.float32)

global_step = tf.contrib.framework.get_or_create_global_step()

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 
                                           decay_steps, decay_rate, staircase=True)
tf.summary.scalar("learning_rate", learning_rate)

tensor_name = "pool3" #"conv3_3/Relu"

with open("vgg16-20160129.tfmodel", mode='rb') as f:
    content = f.read()
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(content)
    
    graph_def = tf.graph_util.extract_sub_graph(graph_def, ["images", tensor_name])
    #graph_def = tf.graph_util.extract_sub_graph(graph_def, ["images", "pool5"])
    tf.import_graph_def(graph_def, input_map={"images": images})
    
del content
graph = tf.get_default_graph()

net = graph.get_tensor_by_name("import/{0}:0".format(tensor_name))
#net = graph.get_tensor_by_name("import/pool5:0")

out_shape = net.get_shape().as_list()
net = tf.stop_gradient(net)

In [ ]:
#graph.get_operations()

In [ ]:
depth = 4096
weights = {
    "Wfc1": tf.Variable(tf.truncated_normal([np.prod(out_shape[1:]), depth]), name="fc1/weights"), 
    "Wfc2": tf.Variable(tf.truncated_normal([depth, depth]), name="fc2/weights"), 
    "Wfc3": tf.Variable(tf.truncated_normal([depth, n_classes]), name="fc3/weights")
} 

biases = {
    "Bfc1": tf.Variable(tf.truncated_normal([depth]), name="fc1/biases"),
    "Bfc2": tf.Variable(tf.truncated_normal([depth]), name="fc2/biases"),
    "Bfc3": tf.Variable(tf.truncated_normal([n_classes]), name="fc3/biases")
}

def inference(net, weigths, biases, keep):
    net = tf.reshape(net, [-1, weights['Wfc1'].get_shape().as_list()[0]], name="fc1/reshape")
    net = tf.add(tf.matmul(net, weights['Wfc1']), biases['Bfc1'], name="fc1/matmul-add")
    net = tf.nn.relu(net, name="fc1/relu")
    net = tf.nn.dropout(net, keep, name="fc1/dropout")

    net = tf.add(tf.matmul(net, weights['Wfc2']), biases['Bfc2'], name="fc2/matmul-add")
    net = tf.nn.relu(net, name="fc2/relu")
    net = tf.nn.dropout(net, keep, name="fc2/dropout")

    net = tf.add(tf.matmul(net, weights['Wfc3']), biases['Bfc3'], name="fc3/matmul-add")
    return net

with tf.name_scope('model'):
    pred = inference(net, weights, biases, keep)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=labels))
    tf.summary.scalar("loss", loss)

with tf.name_scope('sgd'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

with tf.name_scope('accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(labels, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    tf.summary.scalar("accuracy", acc)

In [ ]:

def formatter(d): 
    return 'loss: {0}, acc: {1}%'.format(d.values()[0], d.values()[1]*100)
with tf.device('/cpu:0'):
    hooks = [
        tf.train.StopAtStepHook(num_steps=max_steps),
        tf.train.LoggingTensorHook([loss, acc], every_n_secs=5, formatter=formatter),
        tf.train.NanTensorHook(loss),
    ]

    sess = tf.train.MonitoredTrainingSession(checkpoint_dir=checkpoint_dir,
                                           hooks=hooks,
                                           save_checkpoint_secs=10*60,
                                           save_summaries_steps=summary_step)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    try:
        while not coord.should_stop() or sess.should_stop():
#                 test_accuracy, step = sess.run([acc, global_step],  feed_dict={keep: 01.0})
#                 print("Test Accuracy: %2f" % test_accuracy , "global_step: %d" % int(step//1))
#                 break
                _, step = sess.run([optimizer, global_step], feed_dict={keep: 0.5})
    except tf.errors.OutOfRangeError:
        print('Done training...')
    finally:
        coord.request_stop()

    coord.join(threads)
    sess.close()